In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [2]:
import os
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [3]:
# Define file paths; update these paths if necessary.

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define file paths within your Google Drive
drive_path = '/content/drive/MyDrive/'

train_csv = os.path.join(drive_path, 'dataset_splits/train/train.csv')
val_csv = os.path.join(drive_path, 'dataset_splits/val/val.csv')
test_csv = os.path.join(drive_path, 'dataset_splits/test/test.csv')

data_files = {"train": train_csv, "validation": val_csv, "test": test_csv}
dataset = load_dataset("csv", data_files=data_files)

# Optional: display dataset splits sizes
print("Train samples:", len(dataset["train"]))
print("Validation samples:", len(dataset["validation"]))
print("Test samples:", len(dataset["test"]))


Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Train samples: 3344
Validation samples: 1115
Test samples: 1115


In [4]:
def rename_columns(example):
    return {"text": example["sms"], "label": int(example["label"])}

dataset = dataset.map(rename_columns, remove_columns=dataset["train"].column_names)

# Verify the columns
print("Columns in train split:", dataset["train"].column_names)


Map:   0%|          | 0/3344 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Columns in train split: ['label', 'text']


In [5]:
# Set your model checkpoint
model_checkpoint = "bert-base-uncased"  # Change to another checkpoint if desired

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Tokenize the entire dataset (batched for speed)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Create a data collator for dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3344 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

In [6]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary", pos_label=1)
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [12]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch",  # Add this line to match the evaluation strategy
    report_to="none",  # Disable WandB logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


<ipython-input-12-85d47fc1262a>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.039700,0.053229,0.985650,0.924051,0.973333,0.948052
2,0.012600,0.044116,0.991928,0.979592,0.960000,0.969697
3,0.000300,0.050756,0.991928,0.979592,0.960000,0.969697


TrainOutput(global_step=627, training_loss=0.0372525988829187, metrics={'train_runtime': 172.5982, 'train_samples_per_second': 58.123, 'train_steps_per_second': 3.633, 'total_flos': 312617044870080.0, 'train_loss': 0.0372525988829187, 'epoch': 3.0})

In [ ]:
test_results = trainer.evaluate(tokenized_datasets["test"])
print("Test Set Evaluation Results:")
print(test_results)


In [ ]:
absolute_save_path = "/content/drive/MyDrive/fine_tuned_Bert_Uncased"

trainer.save_model(absolute_save_path)
tokenizer.save_pretrained(absolute_save_path)
print("Fine-tuned bert uncased model and tokenizer saved successfully at", absolute_save_path)